In [1]:
import numpy as np
import cv2
import os
import random
from skimage.io import imread
from skimage.transform import resize
from skimage import feature
from skimage import exposure
import matplotlib.pyplot as plt
from skimage.feature import hog
from sklearn import svm
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
import category_encoders as ce
from sklearn import naive_bayes
import pandas as pd
from sklearn import metrics 
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.base import clone
import matplotlib.pyplot as plt

### Variables y funciones programadas

In [2]:
#Parametros de raiz
raiz=os.getcwd()
print(raiz)

C:\Users\ivana\Desktop\Universidad\4º curso\1º semestre\Vision artificial\Practicas\Practica final\VisionArtificial


In [26]:
#Tamaño de ventana para homogeneizar las imagenes
tamanoVentana = 200

In [4]:
def tratarImagenes(directorio, X_Train, Y_Train, X_Test, Y_Test,num):
    os.chdir(raiz+directorio)
    aux = os.listdir()
    cont = len(aux) * 0.8
    k = 0
    for j in aux:
        img = cv2.imread(j)

        # Convertimos a escala de grises
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        #Aplicamos el filtro de la mediana que es el mejor para eliminar el ruido sal y pimienta
        img = cv2.medianBlur(img,5)
        
        #Aplicamos el filtro gaussiano para eliminar posibles ruidos restantes
        img = cv2.GaussianBlur(img,(5,5),0)

        #Redimensionamos las imagenes para tener en todas el mismo tamaño
        imagen_ampliada = cv2.resize(img, (tamanoVentana,tamanoVentana))
        
        # Detectamos los bordes con Canny
        canny = cv2.Canny(imagen_ampliada, 50, 150)
        
 
        # Buscamos los contornos
        (contornos,_) = cv2.findContours(canny.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
 
        cv2.drawContours(imagen_ampliada,contornos,-1,(0,0,255), 2)
        
        if(k<cont):
            X_Train.append(imagen_ampliada)
            Y_Train.append(num)  
        else:
            X_Test.append(imagen_ampliada)
            Y_Test.append(num)  
        k=k+1

In [5]:
#Parametros para el HOG
pixels_per_cell=(8,8)
cells_per_block=(2,2)
orientations = 9

In [6]:
def HOG(images, pixels_per_cell, cells_per_block):
    lista_hog=[]
    lista_img_hog=[]
    for i,img in enumerate(images):
        carac,img_hog=hog(img, orientations=9, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block,visualize=True, multichannel=False)
        lista_hog.append(carac)
        lista_img_hog.append(img_hog)
    return np.copy(lista_hog),lista_img_hog

# Fase 1: Entrenamiento de un clasificador de monedas

## Fase 1.1: Análisis del conjunto de datos

In [27]:
#Cargamos la imagen y la procesamos para eliminar el ruido
X_Train = []
Y_Train = []
X_Test = []
Y_Test = []

tratarImagenes('\\train\\1c',X_Train, Y_Train, X_Test, Y_Test,1)
tratarImagenes('\\train\\1e',X_Train, Y_Train, X_Test, Y_Test,2)
tratarImagenes('\\train\\2c',X_Train, Y_Train, X_Test, Y_Test,4)
tratarImagenes('\\train\\2e',X_Train, Y_Train, X_Test, Y_Test,5)
tratarImagenes('\\train\\5c',X_Train, Y_Train, X_Test, Y_Test,7)
tratarImagenes('\\train\\10c',X_Train, Y_Train, X_Test, Y_Test,0)
tratarImagenes('\\train\\20c',X_Train, Y_Train, X_Test, Y_Test,3)
tratarImagenes('\\train\\50c',X_Train, Y_Train, X_Test, Y_Test,6)


In [ ]:
print("Tamaño de train") 
len(X_Train)

In [ ]:
print("Tamaño de test") 
len(X_Test)

In [ ]:
#comprobando si se ha aplicado bien el filtro y la redimension

cv2.imshow("imagen",X_Train[456])
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imshow("imagen2",X_Test[100])
cv2.waitKey(0)
cv2.destroyAllWindows()

## Fase 1.2: HOG

In [29]:
X_Train_Coin,_ = HOG(X_Train, pixels_per_cell,cells_per_block)
Y_Train_Coin = np.copy(Y_Train)

X_Test_Coin,_ = HOG(X_Test,pixels_per_cell,cells_per_block)
Y_Test_Coin = np.copy(Y_Test)

# Fase 1.3 Entrenamiento


In [30]:
classifiers = [SVC(), LogisticRegression(random_state=0, max_iter=1000), GaussianNB(), DecisionTreeClassifier()]

clf_names = ['SVM', 'Regr Logistica', 'NB Gaussiano', 'Decision Tree']

score_list = []
aux_params = []
best_score = np.NINF

parameters_dict = {'SVM':{'kernel':('linear','rbf'), 'C':[1,10],'gamma':[0.1,0.001]},
                    'Regr Logistica':{'C':[0.01,1,10,1000] },
                    'NB Gaussiano':{'var_smoothing': np.logspace(0, -9,num=10)},
                    'Decision Tree':{'min_samples_split':[2,3,4], 'criterion':['gini','entropy']},
                 }

for i,(clf_aux,clf_name) in enumerate(zip(classifiers,clf_names)):
        clf = clone(clf_aux)
        clf = GridSearchCV(clf,parameters_dict[clf_name], cv=2, n_jobs=-1)
        clf.fit(X_Train_Coin,Y_Train_Coin)
        
        score=round(clf.score(X_Test_Coin,Y_Test_Coin)*100,2)
        score_list.append(score)
        aux_params.append(clf.best_params_)
        if score >= best_score:
            best_score = score
print(best_score)

72.27


### Mejores parámetros para cada clasificador

### SVM

In [31]:
print("Mejores parámetros para SVM") 
print(aux_params[0])

svc = svm.SVC(kernel='rbf',C=10,gamma=0.001,max_iter=1000)
svc.fit(X_Train_Coin,Y_Train_Coin)
acc = svc.score(X_Train_Coin,Y_Train_Coin)
print("Porcentaje de train: ")
print(acc)

Y_Test_Coin = np.asarray(Y_Test)
acc = svc.score(X_Test_Coin,Y_Test_Coin)
print("Porcentaje de test: ")
print(acc)

Mejores parámetros para SVM
{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
Porcentaje de train: 
1.0
Porcentaje de test: 
0.72265625


### Regresion logística

In [32]:
print("Mejores parámetros para regresion logística") 
print(aux_params[1])

logistic_regression = LogisticRegression(C=10,max_iter=1000)
logistic_regression.fit(X_Train_Coin,Y_Train_Coin)

accRegLog = logistic_regression.score(X_Train_Coin,Y_Train_Coin)
print("Porcentaje de train: ")
print(accRegLog)


Y_Test_Coin = np.asarray(Y_Test)
accRegLogTest = logistic_regression.score(X_Test_Coin,Y_Test_Coin)
print("Porcentaje de test: ")
print(accRegLogTest)


Mejores parámetros para regresion logística
{'C': 10}
Porcentaje de train: 
1.0
Porcentaje de test: 
0.6953125


### NB Gaussiano

In [33]:
print("Mejores parámetros para NB Gaussiano") 
print(aux_params[2])

nb = naive_bayes.GaussianNB(var_smoothing = 0.1)
nb.fit(X_Train_Coin,Y_Train_Coin)
nbscore = nb.score(X_Train_Coin,Y_Train_Coin)
print("Porcentaje de train: ")
print(nbscore)

Y_Test_Coin = np.asarray(Y_Test)
nbscore_test = nb.score(X_Test_Coin,Y_Test_Coin)
print("Porcentaje de test: ")
print(nbscore_test)

Mejores parámetros para NB Gaussiano
{'var_smoothing': 0.1}
Porcentaje de train: 
0.7857829010566763
Porcentaje de test: 
0.57421875


### Decision tree

In [34]:
print("Mejores parámetros para Decision tree") 
print(aux_params[3])

decision_tree = DecisionTreeClassifier(criterion='gini',min_samples_split=2)
decision_tree.fit(X_Train_Coin,Y_Train_Coin)
accDecTree = decision_tree.score(X_Train_Coin,Y_Train_Coin)
print("Porcentaje de train: ")
print(accDecTree)

Y_Test_Coin = np.asarray(Y_Test)
decision_tree_test = logistic_regression.score(X_Test_Coin,Y_Test_Coin)
print("Porcentaje de test: ")
print(decision_tree_test)


Mejores parámetros para Decision tree
{'criterion': 'gini', 'min_samples_split': 2}
Porcentaje de train: 
1.0
Porcentaje de test: 
0.6953125


# Código de generación de csv


In [16]:
def tratarImagenes2(directorio, X_Train,img_labels):
    os.chdir(raiz+directorio)
    aux = os.listdir()
    for j in aux:
        img = cv2.imread(j)

        # Convertimos a escala de grises
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        #Aplicamos el filtro de la mediana que es el mejor para eliminar el ruido sal y pimienta
        img = cv2.medianBlur(img,5)
        
        #Aplicamos el filtro gaussiano para eliminar posibles ruidos restantes
        img = cv2.GaussianBlur(img,(5,5),0)

        #Redimensionamos las imagenes para tener en todas el mismo tamaño
        imagen_ampliada = cv2.resize(img, (tamanoVentana,tamanoVentana))
        
        X_Train.append(imagen_ampliada)
        img_labels.append(j)
        




In [18]:
X_public_test = []
img_test_labels = []


tratarImagenes2('\\public_test',X_public_test, img_test_labels)

X_public_test_Coin,_ = HOG(X_public_test, pixels_per_cell,cells_per_block)



In [20]:
svc = svm.SVC(kernel='rbf',C=10,gamma=0.001,max_iter=10000)
svc.fit(X_Train_Coin,Y_Train_Coin)
y_pred = svc.predict(X_public_test_Coin)
print(y_pred)

[0 6 7 2 2 0 3 2 7 0 2 0 0 0 6 3 0 2 0 0 0 7 3 0 6 2 3 2 0 6 0 0 0 3 7 3 7
 0 3 2 0 0 2 3 2 2 6 0 0 0 0 0 3 6 0 2 0 0 5 7 0 7 0 0 0 0 0 0 0 0 6 3 0 0
 2 4 1 0 7 7 0 0 3 0 7 6 3 0 2 7 0 3 0 6 3 3 3 0 7 7 6 3 3 2 2 7 0 0 0 3 0
 6 0 3 0 6 7 3 3 5 3 0 0 0 7 7 0 6 0 2 2 6 0 0 6 0 0 0 0 0 6 0 0 0 0 3 6 0
 1 0 3 7 2 3 0 1 3 7 3 0 0 7 0 0 3 6 2 6 0 0 0 0 2 0 2 7 7 6 3 0 3 7 0 2 3
 3 0 3 0 7 0 0 0 0 3 0 1 0 7 6 0 0 0 7 0 0 0 1 0 0 6 0 0 3 0 7 2 7 7 7 0 0
 0 2 0 0 0 0 5 3 1 0 2 0 2 0 3 2 0 0 0 0 6 3 0 0 0 7 0 6 3 0 0 7 6 2 0 3 6
 2 6 0 2 0 0 0 0 0 7 0 0 6 0 2 2 2 2 0 0 2 0 3 7 0 0 2 0 2 0 6 0 0 0 2 0 2
 3 7 0 6 6 2 0 0 3 7 7 0 0 0 0 2 0 6 2 0 7 2 3 6 3 0 3 0 0 6 0 2 0 7 0 7 3
 0 2 0 3 7 3 0 7 2 3 0 0 6 0 0 2 7 6 7 0 3 3 2 6 6 0 3 6 3 6 2 0 2 0 0 3 0
 1 0 3 5 0 0 0 3 6 0 6 3 2 2 2 6 0 7 2 3 6 0 0 2 2 2 7 3 0 0 3 0 3 0 7 6 7
 0 6 2 0 0 6 0 6 2 7 2 2 6 0 0 2 0 3 0 6 0 7 0 2 0 6 0 0 3 7 0 0 0 6 3 0 0
 3 3 0 0 2 6 6 2 7 7 3 0 3 0 0 0 3 6 2 0 0 0 7 0 7 3 3 0 3 2 0 6 0 0 3 0 2
 0 1 6 3 2 7 0 0 2 6 4 0 

In [21]:
import csv
fields = ["Id", "Expected"] 
filename = "results.csv"

# writing to csv file 
with open(filename, 'w', newline="") as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile, delimiter=',') 

    # writing the fields 
    csvwriter.writerow(fields) 

    # writing the data rows 
    for i in range(len(y_pred)):
        csvwriter.writerow([img_test_labels[i], y_pred[i]])